In [1]:
# Criar ambiente e sessão spark em memória

# Instalando biblioteca pyspark
!pip install pyspark

In [2]:
# Importando ferramentas (funções) do pyspark
from pyspark.sql import SparkSession as ss
from pyspark.sql import functions as f
from pyspark.sql.types import DoubleType

In [6]:
# Criando objeto spark
spark = ss.builder.appName("SENAI").getOrCreate()

In [7]:
spark

In [3]:
dados = (
    ("ana","analista", 6200),
    ("joao","assistente", 3100),
    ("julio","analista", 7900),
    ("bruna","cordenadora",16000),
    ("pedro","coordenador",15000),
    ("felipe","auxiliar",2500)
)

In [4]:
dados

(('ana', 'analista', 6200),
 ('joao', 'assistente', 3100),
 ('julio', 'analista', 7900),
 ('bruna', 'cordenadora', 16000),
 ('pedro', 'coordenador', 15000),
 ('felipe', 'auxiliar', 2500))

In [5]:
colunas = ("funcionario", "funcao", "salario")

In [8]:
# Criando Data Frame manualmente:
df1 = spark.createDataFrame(dados,colunas)

In [9]:
df1.show()

+-----------+-----------+-------+
|funcionario|     funcao|salario|
+-----------+-----------+-------+
|        ana|   analista|   6200|
|       joao| assistente|   3100|
|      julio|   analista|   7900|
|      bruna|cordenadora|  16000|
|      pedro|coordenador|  15000|
|     felipe|   auxiliar|   2500|
+-----------+-----------+-------+



In [12]:
df1.toPandas()

,funcionario,funcao,salario
0,ana,analista,6200
1,joao,assistente,3100
2,julio,analista,7900
3,bruna,cordenadora,16000
4,pedro,coordenador,15000
5,felipe,auxiliar,2500


In [14]:
dados = (
    ("claudio","analista", 5800),
    ("beatriz","assistente", 3300)
)

In [15]:
df2 = spark.createDataFrame(dados,colunas)

In [16]:
df2.show()

+-----------+----------+-------+
|funcionario|    funcao|salario|
+-----------+----------+-------+
|    claudio|  analista|   5800|
|    beatriz|assistente|   3300|
+-----------+----------+-------+



In [17]:
df = df1.union(df2)

In [18]:
df.show()

+-----------+-----------+-------+
|funcionario|     funcao|salario|
+-----------+-----------+-------+
|        ana|   analista|   6200|
|       joao| assistente|   3100|
|      julio|   analista|   7900|
|      bruna|cordenadora|  16000|
|      pedro|coordenador|  15000|
|     felipe|   auxiliar|   2500|
|    claudio|   analista|   5800|
|    beatriz| assistente|   3300|
+-----------+-----------+-------+



In [19]:
# Exibir informações sobre os tipos de dados e nomes e colunas do dataframe:
df.printSchema()

root
 |-- funcionario: string (nullable = true)
 |-- funcao: string (nullable = true)
 |-- salario: long (nullable = true)



In [21]:
# Adicionando nova coluna no dataframe spark:
# Metodo de visualização de uma nova coluna tabela view
df.withColumn("cenario1",
              f.round(df.salario * 1.05, 2)).show()

+-----------+-----------+-------+--------+
|funcionario|     funcao|salario|cenario1|
+-----------+-----------+-------+--------+
|        ana|   analista|   6200|  6510.0|
|       joao| assistente|   3100|  3255.0|
|      julio|   analista|   7900|  8295.0|
|      bruna|cordenadora|  16000| 16800.0|
|      pedro|coordenador|  15000| 15750.0|
|     felipe|   auxiliar|   2500|  2625.0|
|    claudio|   analista|   5800|  6090.0|
|    beatriz| assistente|   3300|  3465.0|
+-----------+-----------+-------+--------+



In [25]:
# Efetivando a alteração do df
df = df.withColumn("cenario1",
              f.round(df.salario * 1.05, 2))

In [26]:
df.toPandas()

,funcionario,funcao,salario,cenario1
0,ana,analista,6200,6510.0
1,joao,assistente,3100,3255.0
2,julio,analista,7900,8295.0
3,bruna,cordenadora,16000,16800.0
4,pedro,coordenador,15000,15750.0
5,felipe,auxiliar,2500,2625.0
6,claudio,analista,5800,6090.0
7,beatriz,assistente,3300,3465.0


In [28]:
# Criando coluna condicional
# No novo cenário será aplicado um reajuste de 7% para quem ganha
# ate 5000.  Para outros salários será o reajuste padrão de 5%

df.withColumn("cenario2",
              f.when(df.salario <= 5000, df.salario * 1.07)\
              .otherwise(df.salario *1.05)).show()

+-----------+-----------+-------+--------+--------+
|funcionario|     funcao|salario|cenario1|cenario2|
+-----------+-----------+-------+--------+--------+
|        ana|   analista|   6200|  6510.0|  6510.0|
|       joao| assistente|   3100|  3255.0|  3317.0|
|      julio|   analista|   7900|  8295.0|  8295.0|
|      bruna|cordenadora|  16000| 16800.0| 16800.0|
|      pedro|coordenador|  15000| 15750.0| 15750.0|
|     felipe|   auxiliar|   2500|  2625.0|  2675.0|
|    claudio|   analista|   5800|  6090.0|  6090.0|
|    beatriz| assistente|   3300|  3465.0|  3531.0|
+-----------+-----------+-------+--------+--------+



In [29]:
# Efetivando alterção
df = df.withColumn("cenario2",
              f.when(df.salario <= 5000, df.salario * 1.07)\
              .otherwise(df.salario *1.05))

In [ ]:
df.toPandas()

In [32]:
# Filtrando dados:
df.where("salario >= 7000").show()

+-----------+-----------+-------+--------+--------+
|funcionario|     funcao|salario|cenario1|cenario2|
+-----------+-----------+-------+--------+--------+
|      julio|   analista|   7900|  8295.0|  8295.0|
|      bruna|cordenadora|  16000| 16800.0| 16800.0|
|      pedro|coordenador|  15000| 15750.0| 15750.0|
+-----------+-----------+-------+--------+--------+



In [33]:
df.where("funcao like '%coord%'").show()

+-----------+-----------+-------+--------+--------+
|funcionario|     funcao|salario|cenario1|cenario2|
+-----------+-----------+-------+--------+--------+
|      pedro|coordenador|  15000| 15750.0| 15750.0|
+-----------+-----------+-------+--------+--------+



In [34]:
# Importando dataframe .csv
# inferSchema: sobe o dados das variaveis certas
url = "/content/drive/MyDrive/BasesPark/empresas/part*"
empresas = spark.read.csv(
    url,
    sep=";",
    header=True,
    inferSchema=True,
    encoding="latin1"

)

In [35]:
empresas.show()

+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
| cnpj|        razao_social|natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+-----+--------------------+-----------------+------------------------+--------------+-----+---------------+
|  306|FRANCAMAR REFRIGE...|             2240|                      49|          0,00|    1|           NULL|
| 1355|BRASILEIRO & OLIV...|             2062|                      49|          0,00|    5|           NULL|
| 4820|REGISTRO DE IMOVE...|             3034|                      32|          0,00|    5|           NULL|
| 5347|ROSELY APARECIDA ...|             2135|                      50|          0,00|    5|           NULL|
| 6846|BADU E FILHOS TEC...|             2062|                      49|       4000,00|    1|           NULL|
| 8416|  ELETRICA RUBI LTDA|             2062|                      49|          0,00|    5|           NULL|
| 8992|SHIROMA VEIC

In [36]:
empresas.printSchema()

root
 |-- cnpj: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_responsavel: integer (nullable = true)
 |-- capital_social: string (nullable = true)
 |-- porte: integer (nullable = true)
 |-- ente_federativo: string (nullable = true)



In [37]:
# Usando o toPandas() com muitas linhas
empresas.limit(20).toPandas()

,cnpj,razao_social,natureza_juridica,qualificacao_responsavel,capital_social,porte,ente_federativo
0,306,FRANCAMAR REFRIGERACAO TECNICA S/C LTDA,2240,49,"0,00",1,None
1,1355,BRASILEIRO & OLIVEIRA LTDA,2062,49,"0,00",5,None
2,4820,"REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E ...",3034,32,"0,00",5,None
3,5347,ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS,2135,50,"0,00",5,None
4,6846,BADU E FILHOS TECIDOS LTDA,2062,49,"4000,00",1,None
5,8416,ELETRICA RUBI LTDA,2062,49,"0,00",5,None
6,8992,SHIROMA VEICULOS LTDA.,2062,49,"0,00",5,None
7,9091,CONTATOS BAR E LANCHONETE LTDA,2062,49,"0,00",5,None
8,9614,ANTONIA APARECIDA DE SOUZA ULIANA,2135,50,"0,00",5,None
9,9896,DORACY CORAT DA COSTA,2135,50,"0,00",5,None


In [40]:
# QTD de registros
print(f"{empresas.count():,.2f}")

4,585,679.00


In [41]:
empresas.show(truncate=False)

+-----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|cnpj |razao_social                                                                                |natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+-----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|306  |FRANCAMAR REFRIGERACAO TECNICA S/C LTDA                                                     |2240             |49                      |0,00          |1    |NULL           |
|1355 |BRASILEIRO & OLIVEIRA LTDA                                                                  |2062             |49                      |0,00          |5    |NULL           |
|4820 |REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E TABELIONATO E REGISTRO DE CONSTRATOS MARIT

In [43]:
empresas.show(5, False)

+----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|cnpj|razao_social                                                                                |natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+----+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|306 |FRANCAMAR REFRIGERACAO TECNICA S/C LTDA                                                     |2240             |49                      |0,00          |1    |NULL           |
|1355|BRASILEIRO & OLIVEIRA LTDA                                                                  |2062             |49                      |0,00          |5    |NULL           |
|4820|REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E TABELIONATO E REGISTRO DE CONSTRATOS MARITIMOS|3

In [46]:
# Mostre alguns registros que tenha a palavra "hospital" em sua razão social:
# O 'like' diferencia maiúsculas de minúsculas
# O 'ilike' não diferencia

empresas.where("razao_social ilike '%hospital%'").show(15, False)

+--------+-------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|cnpj    |razao_social                                                             |natureza_juridica|qualificacao_responsavel|capital_social|porte|ente_federativo|
+--------+-------------------------------------------------------------------------+-----------------+------------------------+--------------+-----+---------------+
|1756451 |AUTOMED DIST DE PRODUTOS AUTOMOTIVOS E HOSPITALARES LTDA                 |2062             |49                      |0,00          |1    |NULL           |
|2616076 |SERVICO MEDICO HOSPITALAR DE TAGUATINGA LTDA                             |2062             |49                      |0,00          |1    |NULL           |
|5082994 |LEBEC MEDICAL COMERCIO E SERVICOS DE SISTEMAS HOSPITALARES LTDA          |2062             |49                      |80000,00      |1    |NULL           |
|5791196 |

In [47]:
# Renomeando colunas:
empresas.withColumnsRenamed({
    "natureza_juridica": "natureza",
    "qualificacao_responsavel": "responsavel",
    "capital_social": "capital",
    "ente_federativo": "ente"
}).show()

+-----+--------------------+--------+-----------+-------+-----+----+
| cnpj|        razao_social|natureza|responsavel|capital|porte|ente|
+-----+--------------------+--------+-----------+-------+-----+----+
|  306|FRANCAMAR REFRIGE...|    2240|         49|   0,00|    1|NULL|
| 1355|BRASILEIRO & OLIV...|    2062|         49|   0,00|    5|NULL|
| 4820|REGISTRO DE IMOVE...|    3034|         32|   0,00|    5|NULL|
| 5347|ROSELY APARECIDA ...|    2135|         50|   0,00|    5|NULL|
| 6846|BADU E FILHOS TEC...|    2062|         49|4000,00|    1|NULL|
| 8416|  ELETRICA RUBI LTDA|    2062|         49|   0,00|    5|NULL|
| 8992|SHIROMA VEICULOS ...|    2062|         49|   0,00|    5|NULL|
| 9091|CONTATOS BAR E LA...|    2062|         49|   0,00|    5|NULL|
| 9614|ANTONIA APARECIDA...|    2135|         50|   0,00|    5|NULL|
| 9896|DORACY CORAT DA C...|    2135|         50|   0,00|    5|NULL|
|12112|LANCHONETE RIO VE...|    2062|         49|   0,00|    5|NULL|
|12605|VALMAR JACAREI CO...|    20

In [48]:
# Efetivando alteração:

empresas = empresas.withColumnsRenamed({
    "natureza_juridica": "natureza",
    "qualificacao_responsavel": "responsavel",
    "capital_social": "capital",
    "ente_federativo": "ente"
})

In [49]:
empresas.show(5, False)

+----+--------------------------------------------------------------------------------------------+--------+-----------+-------+-----+----+
|cnpj|razao_social                                                                                |natureza|responsavel|capital|porte|ente|
+----+--------------------------------------------------------------------------------------------+--------+-----------+-------+-----+----+
|306 |FRANCAMAR REFRIGERACAO TECNICA S/C LTDA                                                     |2240    |49         |0,00   |1    |NULL|
|1355|BRASILEIRO & OLIVEIRA LTDA                                                                  |2062    |49         |0,00   |5    |NULL|
|4820|REGISTRO DE IMOVEIS, TABELIONATO 1 DE NOTAS E TABELIONATO E REGISTRO DE CONSTRATOS MARITIMOS|3034    |32         |0,00   |5    |NULL|
|5347|ROSELY APARECIDA MONTEIRO CALTABIANO FREITAS                                                |2135    |50         |0,00   |5    |NULL|
|6846|BADU E FILHOS 

In [53]:
# Antes de converter a coluna 'capital' teremos que substituir a ',' por '.':
# Caso contrário o pyspark não consegue converter a coluna... ela ficará nula.

empresas.withColumn("capital",
                    f.regexp_replace("capital", ",", ".",)).show()

+-----+--------------------+--------+-----------+-------+-----+----+
| cnpj|        razao_social|natureza|responsavel|capital|porte|ente|
+-----+--------------------+--------+-----------+-------+-----+----+
|  306|FRANCAMAR REFRIGE...|    2240|         49|   0.00|    1|NULL|
| 1355|BRASILEIRO & OLIV...|    2062|         49|   0.00|    5|NULL|
| 4820|REGISTRO DE IMOVE...|    3034|         32|   0.00|    5|NULL|
| 5347|ROSELY APARECIDA ...|    2135|         50|   0.00|    5|NULL|
| 6846|BADU E FILHOS TEC...|    2062|         49|4000.00|    1|NULL|
| 8416|  ELETRICA RUBI LTDA|    2062|         49|   0.00|    5|NULL|
| 8992|SHIROMA VEICULOS ...|    2062|         49|   0.00|    5|NULL|
| 9091|CONTATOS BAR E LA...|    2062|         49|   0.00|    5|NULL|
| 9614|ANTONIA APARECIDA...|    2135|         50|   0.00|    5|NULL|
| 9896|DORACY CORAT DA C...|    2135|         50|   0.00|    5|NULL|
|12112|LANCHONETE RIO VE...|    2062|         49|   0.00|    5|NULL|
|12605|VALMAR JACAREI CO...|    20

In [54]:
empresas = empresas.withColumn("capital",
                    f.regexp_replace("capital", ",", ".",))

In [55]:
empresas.show()

+-----+--------------------+--------+-----------+-------+-----+----+
| cnpj|        razao_social|natureza|responsavel|capital|porte|ente|
+-----+--------------------+--------+-----------+-------+-----+----+
|  306|FRANCAMAR REFRIGE...|    2240|         49|   0.00|    1|NULL|
| 1355|BRASILEIRO & OLIV...|    2062|         49|   0.00|    5|NULL|
| 4820|REGISTRO DE IMOVE...|    3034|         32|   0.00|    5|NULL|
| 5347|ROSELY APARECIDA ...|    2135|         50|   0.00|    5|NULL|
| 6846|BADU E FILHOS TEC...|    2062|         49|4000.00|    1|NULL|
| 8416|  ELETRICA RUBI LTDA|    2062|         49|   0.00|    5|NULL|
| 8992|SHIROMA VEICULOS ...|    2062|         49|   0.00|    5|NULL|
| 9091|CONTATOS BAR E LA...|    2062|         49|   0.00|    5|NULL|
| 9614|ANTONIA APARECIDA...|    2135|         50|   0.00|    5|NULL|
| 9896|DORACY CORAT DA C...|    2135|         50|   0.00|    5|NULL|
|12112|LANCHONETE RIO VE...|    2062|         49|   0.00|    5|NULL|
|12605|VALMAR JACAREI CO...|    20

In [56]:
empresas.printSchema()

root
 |-- cnpj: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- natureza: integer (nullable = true)
 |-- responsavel: integer (nullable = true)
 |-- capital: string (nullable = true)
 |-- porte: integer (nullable = true)
 |-- ente: string (nullable = true)



In [58]:
# Convertendo coluna de string para numérica
empresas.withColumn("capital",
                    empresas.capital.cast(DoubleType())).show()

+-----+--------------------+--------+-----------+-------+-----+----+
| cnpj|        razao_social|natureza|responsavel|capital|porte|ente|
+-----+--------------------+--------+-----------+-------+-----+----+
|  306|FRANCAMAR REFRIGE...|    2240|         49|    0.0|    1|NULL|
| 1355|BRASILEIRO & OLIV...|    2062|         49|    0.0|    5|NULL|
| 4820|REGISTRO DE IMOVE...|    3034|         32|    0.0|    5|NULL|
| 5347|ROSELY APARECIDA ...|    2135|         50|    0.0|    5|NULL|
| 6846|BADU E FILHOS TEC...|    2062|         49| 4000.0|    1|NULL|
| 8416|  ELETRICA RUBI LTDA|    2062|         49|    0.0|    5|NULL|
| 8992|SHIROMA VEICULOS ...|    2062|         49|    0.0|    5|NULL|
| 9091|CONTATOS BAR E LA...|    2062|         49|    0.0|    5|NULL|
| 9614|ANTONIA APARECIDA...|    2135|         50|    0.0|    5|NULL|
| 9896|DORACY CORAT DA C...|    2135|         50|    0.0|    5|NULL|
|12112|LANCHONETE RIO VE...|    2062|         49|    0.0|    5|NULL|
|12605|VALMAR JACAREI CO...|    20

In [59]:
empresas = empresas.withColumn("capital",
                    empresas.capital.cast(DoubleType()))

In [60]:
empresas.printSchema()

root
 |-- cnpj: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- natureza: integer (nullable = true)
 |-- responsavel: integer (nullable = true)
 |-- capital: double (nullable = true)
 |-- porte: integer (nullable = true)
 |-- ente: string (nullable = true)



In [61]:
empresas.show(3)

+----+--------------------+--------+-----------+-------+-----+----+
|cnpj|        razao_social|natureza|responsavel|capital|porte|ente|
+----+--------------------+--------+-----------+-------+-----+----+
| 306|FRANCAMAR REFRIGE...|    2240|         49|    0.0|    1|NULL|
|1355|BRASILEIRO & OLIV...|    2062|         49|    0.0|    5|NULL|
|4820|REGISTRO DE IMOVE...|    3034|         32|    0.0|    5|NULL|
+----+--------------------+--------+-----------+-------+-----+----+
only showing top 3 rows



In [62]:
empresas.where("capital is null").show()

+----+------------+--------+-----------+-------+-----+----+
|cnpj|razao_social|natureza|responsavel|capital|porte|ente|
+----+------------+--------+-----------+-------+-----+----+
+----+------------+--------+-----------+-------+-----+----+



In [63]:
empresas.where("ente is not null").show(20, False)

+--------+----------------------------------------------------------------------------------------------+--------+-----------+-------+-----+---------------------------+
|cnpj    |razao_social                                                                                  |natureza|responsavel|capital|porte|ente                       |
+--------+----------------------------------------------------------------------------------------------+--------+-----------+-------+-----+---------------------------+
|784274  |SECRETARIA MUNICIPAL DE SAUDE                                                                 |1031    |5          |0.0    |5    |PARATINGA - BA             |
|2030715 |AGENCIA NACIONAL DE TELECOMUNICACOES                                                          |1104    |16         |0.0    |5    |UNIï¿½O                    |
|3104818 |AUTARQUIA MUNICIPAL DE ESPORTES E RECREACAO DE OURINHOS                                       |1120    |16         |0.0    |5    |OURINHOS - SP  

In [65]:
empresas.where("ente ilike'%sao jose do rio preto%'").show(50, False)

+-------+-------------------------------------------+--------+-----------+-------+-----+--------------------------+
|cnpj   |razao_social                               |natureza|responsavel|capital|porte|ente                      |
+-------+-------------------------------------------+--------+-----------+-------+-----+--------------------------+
|4691691|SERVICO MUNICIPAL AUTONOMO DE AGUA E ESGOTO|1120    |16         |0.0    |5    |SAO JOSE DO RIO PRETO - SP|
+-------+-------------------------------------------+--------+-----------+-------+-----+--------------------------+



In [67]:
# Empresas com capital etre 50 e 100k
empresas.where("capital between 50000 and 100000").show(20, False)

+-------+--------------------------------------------------------------+--------+-----------+--------+-----+----+
|cnpj   |razao_social                                                  |natureza|responsavel|capital |porte|ente|
+-------+--------------------------------------------------------------+--------+-----------+--------+-----+----+
|116791 |AGNALDO ROCHA VIANA                                           |2135    |50         |60000.0 |1    |NULL|
|399680 |MARIANTE COMERCIO DE ALIMENTOS LTDA                           |2062    |49         |50000.0 |5    |NULL|
|550169 |FERNANDO SKAF                                                 |2135    |50         |70000.0 |3    |NULL|
|616948 |NOVA PRIMUS COMERCIO DE PECAS INDUSTRIAL LTDA                 |2062    |49         |50000.0 |1    |NULL|
|655784 |HOB & NOB COM. IMPORT. E EXPORT. LTDA                         |2062    |49         |52000.0 |3    |NULL|
|820160 |I B ABRAMOSKI EIRELI                                          |2305    |65     

In [73]:
# Normalizando dados nulos
# FILLna ele renomeia o dado
empresas = empresas.fillna({
    "ente":"N/A",
    "capital":0
})

In [74]:
empresas.show(3)

+----+--------------------+--------+-----------+-------+-----+----+
|cnpj|        razao_social|natureza|responsavel|capital|porte|ente|
+----+--------------------+--------+-----------+-------+-----+----+
| 306|FRANCAMAR REFRIGE...|    2240|         49|    0.0|    1| N/A|
|1355|BRASILEIRO & OLIV...|    2062|         49|    0.0|    5| N/A|
|4820|REGISTRO DE IMOVE...|    3034|         32|    0.0|    5| N/A|
+----+--------------------+--------+-----------+-------+-----+----+
only showing top 3 rows



In [76]:
# Importando dados de estabelecimentos:
url = "/content/drive/MyDrive/BasesPark/estabelecimentos/part*"
estabs = spark.read.csv(
    url,
    sep=";",
    inferSchema=True,
    header=True,
    encoding="latin1"
)

In [77]:
estabs.count()

4836219

In [78]:
estabs.show()

+-----------+----------+-------+---------------------------+--------------------+------------------+-----------------------+-------------------------+--------------------------+----+------------------------+---------------------+----------------------+------------------+--------------------+------+-------------------+------------------+--------+---+---------+-----+----------+-----+----------+----------+--------+--------------------+-----------------+-------------------------+
|cnpj_basico|cnpj_ordem|cnpj_dv|identificador_matriz_filial|       nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_da_cidade_no_exterior|pais|data_de_inicio_atividade|cnae_fiscal_principal|cnae_fiscal_secundaria|tipo_de_logradouro|          logradouro|numero|        complemento|            bairro|     cep| uf|municipio|ddd_1|telefone_1|ddd_2|telefone_2|ddd_do_fax|     fax|  correio_eletronico|situacao_especial|data_da_situacao_especial|
+-----------+----------+-------+------